# 生成组合特征

## 得到不含user_name特征的PR（输入：f_count:第几个特征（为了输出命名），feature_user：拥有该特征的用户列表；find_user：用该特征所得到的block中的用户列表）

In [49]:
import pandas as pd
import csv
import numpy
def get_no_name_PR(f_count,feature_user,find_user):
    
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(find_user)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    #raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    #raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(feature_user)][['user_name','label']]#原始数据中坏用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    #if(TP==0):
        #return 0
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算

    feature_label = dataset[['user_name','label']]
    feature_label = feature_label.drop_duplicates()#去重
    feature_label.loc[:,'label']=np.nan
    #feature_label.rename(columns={'label':'feature'+str(f_count)+"block_"+str(choice)+'_label'})
    #feature_label=pd.DataFrame(feature_label,index=range(len(feature_label)),columns=['feature'+str(f_count)+"/block_"+str(choice)])
    feature_label.loc[feature_label['user_name'].isin(feature_user),'label']=0
    feature_label.loc[feature_label['user_name'].isin(find_user),'label']=1
    feature_label.rename(columns={'label':str(f_count)},inplace=True)
    
    return TP,FP,Precision,feature_label
    print("TP",TP,"P",P,"FP",FP,"N",N)
    print('测试特征'+f_count)
    print('\t\t精确率 Precision\t= '+str(Precision))
    print('\t\t召回率 Recall   \t= '+str(Recall))

  

## 得到有user_name的特征的PR(输入：f_count：第几个特征（为了读取特征文件），j:无用，choice:第几个block)

In [44]:
import pandas as pd
import csv
import numpy
def get_feature_label(f_count,j,choice):#f_count：the index of feature;j: index of test; choice: index of block
    
    #准备用于读取数据的 pandas头（防止第一行数据成为标签）
    data_path="/data/jupyter_root/dcube_data/raw_have_name/feature"#feature文件位置
    try:
        cols = pd.read_csv(data_path+str(f_count)+"/block_1.tuples",nrows=1).columns#先读第一行，用于提取数据维度
    except:
        print("false1")
        return 0,0,0        

    user_index=['user_name']#初始化一个等待补全的names
    #print(cols.shape[0]-1)
    for i in range(1,cols.shape[0]-1):
            user_index.append(str(i))
    user_index.append('count')#补上最后的count
    
    #开始读取数据
    #result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block_1.tuples",sep=',',names=user_index)#读取全部的数据，并给上names
    #try:
    if(choice=='all'):#读取blocks.txt
        result=pd.read_csv(data_path+str(f_count)+"/blocks.txt",sep=',',names=user_index)
    elif(choice==1 or choice==2 or choice==3 or choice==4 or choice==5 ):#读取block_choice.tuples
        result=pd.read_csv(data_path+str(f_count)+"/block_"+str(choice)+".tuples",sep=',',names=user_index)
    elif(choice==12 or choice==13 or choice==23 or choice ==123 ):#读取blockchoice.txt
        result=pd.read_csv(data_path+str(f_count)+"/block"+str(choice)+".txt",sep=',',names=user_index)
    #except:
    #    print("false2")
    #    return 0
    names = result['user_name'].drop_duplicates()#预测出坏用户的用户列表
    #print(names)
    
    dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
    data = dataset[dataset['user_name'].isin(names)][['user_name','label']]#预测出坏用户的真实标签分布
    data = data.drop_duplicates()#去重
    #print(data)

    
    raw_result=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    raw_names = raw_result['user_name'].drop_duplicates()#原始数据中用户的用户列表
    #print(raw_names)
    raw_data = dataset[dataset['user_name'].isin(raw_names)][['user_name','label']]#原始数据中用户的真实标签分布
    raw_data = raw_data.drop_duplicates()#去重
    #print(raw_data)
    
    
    TP=data.label.sum()
    P=raw_data.label.sum()
    FP=data.label.count()-data.label.sum()
    N=raw_data.label.count()-raw_data.label.sum()
    Precision = (TP/P)/(TP/P+FP/N) #精确率计算

    feature_label = dataset[['user_name','label']]
    feature_label = feature_label.drop_duplicates()#去重
    feature_label.loc[:,'label']=np.nan
    #feature_label.rename(columns={'label':'feature'+str(f_count)+"block_"+str(choice)+'_label'})
    #feature_label=pd.DataFrame(feature_label,index=range(len(feature_label)),columns=['feature'+str(f_count)+"/block_"+str(choice)])
    feature_label.loc[feature_label['user_name'].isin(raw_names),'label']=0
    feature_label.loc[feature_label['user_name'].isin(names),'label']=1
    feature_label.rename(columns={'label':'have_name_feature'+str(f_count)+"block_"+str(choice)+'_label'},inplace=True)
    
    
    #data_label_1 = dataset[dataset['label']==1][['user_name','label']] #原数据中label为1的坏用户
    #data_label_1 = data_label_1.drop_duplicates()#去重
    Recall = data.label.sum()/raw_data.label.sum() # 召回率计算
    #print("TP",TP,"P",P,"FP",FP,"N",N)
    #print('测试特征'+str(f_count)+"_"+str(j)+str(choice))
    #print('\t\t精确率 Precision\t= '+str(Precision))
    #print('\t\t召回率 Recall   \t= '+str(Recall))
    return TP,FP,Precision,feature_label

## 读取所选择的特征并组合在一个datafarme中 

In [50]:
import pandas as pd
import csv
import numpy
import re

def get_set(s):
    s=re.sub('[ {}]','',s).split(',')
    def func(x):
        return int(float(x))
    return set(map(func,s))

dataset = pd.read_csv(open('/data/csv/label_eventsV1.csv','r',encoding = 'gb18030'))#读取总标签数据
user_and_label = dataset[['user_name','label']]#预测出坏用户的真实标签分布
user_and_label = user_and_label .drop_duplicates()#去重

#读取no_name特征
result_list=pd.read_csv("/data/jupyter_root/dcube_data/raw_no_name/result_list.csv")
for i in range(12):
    al=get_set(result_list.iloc[i,0])
    #try:
    TP,FP,Precision,feature_label=get_no_name_PR('no_name_feature'+str(i)+'block_1_label',al,get_set(result_list.iloc[i,1]))
    if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    #except :
        #continue
    try:
        TP,FP,Precision,feature_label=get_no_name_PR('no_name_feature'+str(i)+'block_2_label',al,get_set(result_list.iloc[i,2]))
        if(TP>5 and  (Precision>0.53)) or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
        continue
    try:
        TP,FP,Precision,feature_label=get_no_name_PR('no_name_feature'+str(i)+'block_3_label',al,get_set(result_list.iloc[i,3]))
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
        continue
    try:
        TP,FP,Precision,feature_label=get_no_name_PR('no_name_feature'+str(i)+'block_4_label',al,get_set(result_list.iloc[i,4]))
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
        continue
    try:
        TP,FP,Precision,feature_label=get_no_name_PR('no_name_feature'+str(i)+'block_5_label',al,get_set(result_list.iloc[i,5]))
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
        continue

# #读取no_name_add特征
# result_list=pd.read_csv("/data/jupyter_root/dcube_data/no_name_add/result_list.csv")
# for i in [40,64,56,140,149,127]:
#     al=get_set(result_list.iloc[i,0])
#     #try:
#     TP,FP,Precision,feature_label=get_no_name_PR("feature_"+str(i)+'block_1',al,get_set(result_list.iloc[i,1]))
#     if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
#             user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#     #except :
#         #continue
#     try:
#         TP,FP,Precision,feature_label=get_no_name_PR("feature_"+str(i)+'block_2',al,get_set(result_list.iloc[i,2]))
#         if(TP>5 and  (Precision>0.53)) or (FP>5 and  (Precision<0.47)) :
#             user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#     except :
#         continue
#     try:
#         TP,FP,Precision,feature_label=get_no_name_PR("feature_"+str(i)+'block_3',al,get_set(result_list.iloc[i,3]))
#         if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
#             user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#     except :
#         continue

# #读取no_name_add的额外特征  
# result_list=pd.read_csv("/data/jupyter_root/dcube_data/no_name_add/result_list2_811.csv")
# for i in [3]:
#     al=get_set(result_list.iloc[i,0])
#     #try:
#     TP,FP,Precision,feature_label=get_no_name_PR("feature_"+str(i)+'block_1',al,get_set(result_list.iloc[i,1]))
#     if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
#             user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#     #except :
#         #continue
#     try:
#         TP,FP,Precision,feature_label=get_no_name_PR("feature_"+str(i)+'block_2',al,get_set(result_list.iloc[i,2]))
#         if(TP>5 and  (Precision>0.53)) or (FP>5 and  (Precision<0.47)) :
#             user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#     except :
#         continue
#     try:
#         TP,FP,Precision,feature_label=get_no_name_PR("feature_"+str(i)+'block_3',al,get_set(result_list.iloc[i,3]))
#         if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
#             user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#     except :
#         continue

#读取含user_name的特征
for i in range(1,10):
    j=1
    try:
        TP,FP,Precision,feature_label=get_feature_label(i,j,1)
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
        continue
    try:
        TP,FP,Precision,feature_label=get_feature_label(i,j,2)
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
        continue
    try:
        TP,FP,Precision,feature_label=get_feature_label(i,j,3)
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
           continue
    try:
        TP,FP,Precision,feature_label=get_feature_label(i,j,4)
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
           continue
    try:
        TP,FP,Precision,feature_label=get_feature_label(i,j,5)
        if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
            user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    except :
           continue

#读取含user_name的cout>1的特征
# for i in [97,101]:
#  j=1
#  try:
#      TP,FP,Precision,feature_label=get_feature_label_2(i,j,1)
#      if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
#          user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#  except :
#      continue
#  try:
#      TP,FP,Precision,feature_label=get_feature_label_2(i,j,2)
#      if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
#          user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#  except :
#      continue
#  try:
#      TP,FP,Precision,feature_label=get_feature_label_2(i,j,3)
#      if(TP>5 and  (Precision>0.53))or (FP>5 and  (Precision<0.47)) :
#          user_and_label=user_and_label.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
#  except :
#      continue

#读取含user_name的一维度的add的特征
for i in range(10,20):
    cols = pd.read_csv('/data/jupyter_root/dcube_data/raw_have_name/feature'+str(i)+'.txt',nrows=1).columns
    user_index=['user_name']#初始化一个等待补全的names
    #print(cols.shape[0]-1)
    #for i in range(1,cols.shape[0]-1):
            #user_index.append(str(i))
    user_index.append('have_name_feature'+str(i)+'count')#补上最后的count
    oneadd=pd.read_csv('/data/jupyter_root/dcube_data/raw_have_name/feature'+str(i)+'.txt',sep=',',names=user_index)
    user_and_label=user_and_label.merge(oneadd,left_on='user_name',right_on='user_name',how='left')

user_and_label.to_csv('/data/jupyter_root/LSK/raw_feature_block_label.csv',index=None,header=True)
user_and_label

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars


,user_name,label,no_name_feature0block_2_label,no_name_feature1block_2_label,no_name_feature1block_3_label,no_name_feature1block_4_label,no_name_feature1block_5_label,no_name_feature2block_1_label,no_name_feature3block_1_label,no_name_feature3block_2_label,...,have_name_feature10count,have_name_feature11count,have_name_feature12count,have_name_feature13count,have_name_feature14count,have_name_feature15count,have_name_feature16count,have_name_feature17count,have_name_feature18count,have_name_feature19count
0,114386,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1522376,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,3.0,3.0,3.0,4.0,2.0,1.0,1.0,1.0
2,74965,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,4.0,5.0,7.0,7.0,1.0,12.0,5.0,10.0
3,551978,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0
4,300417,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
5,1561348,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0
6,194233,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,4.0
7,1619824,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0
8,69893,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0
9,481152,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [51]:
user_and_label.columns

Index(['user_name', 'label', 'no_name_feature0block_2_label',
       'no_name_feature1block_2_label', 'no_name_feature1block_3_label',
       'no_name_feature1block_4_label', 'no_name_feature1block_5_label',
       'no_name_feature2block_1_label', 'no_name_feature3block_1_label',
       'no_name_feature3block_2_label', 'no_name_feature3block_3_label',
       'no_name_feature3block_4_label', 'no_name_feature3block_5_label',
       'no_name_feature4block_1_label', 'no_name_feature4block_2_label',
       'no_name_feature4block_5_label', 'no_name_feature5block_1_label',
       'no_name_feature5block_2_label', 'no_name_feature5block_3_label',
       'no_name_feature5block_4_label', 'no_name_feature6block_1_label',
       'no_name_feature6block_2_label', 'no_name_feature6block_3_label',
       'no_name_feature7block_1_label', 'no_name_feature7block_2_label',
       'no_name_feature7block_3_label', 'no_name_feature7block_5_label',
       'no_name_feature8block_1_label', 'no_name_feature8block

## 去除空行

In [52]:
import pandas as pd
user_and_label=user_and_label.drop(range(9409,9502))
user_and_label.to_csv('/data/jupyter_root/LSK/raw_feature_block_label.csv',index=None,header=True)
user_and_label

,user_name,label,no_name_feature0block_2_label,no_name_feature1block_2_label,no_name_feature1block_3_label,no_name_feature1block_4_label,no_name_feature1block_5_label,no_name_feature2block_1_label,no_name_feature3block_1_label,no_name_feature3block_2_label,...,have_name_feature10count,have_name_feature11count,have_name_feature12count,have_name_feature13count,have_name_feature14count,have_name_feature15count,have_name_feature16count,have_name_feature17count,have_name_feature18count,have_name_feature19count
0,114386,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1522376,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,3.0,3.0,3.0,4.0,2.0,1.0,1.0,1.0
2,74965,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,4.0,5.0,7.0,7.0,1.0,12.0,5.0,10.0
3,551978,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0
4,300417,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
5,1561348,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0
6,194233,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,4.0
7,1619824,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0
8,69893,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0
9,481152,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


## 生成十则交叉验证的测试集和训练集 

In [12]:
from sklearn.model_selection import ShuffleSplit, GroupShuffleSplit
x_train_names_all = np.array(user_and_label['user_name'])
y_train_labels_all = np.array(user_and_label['label'])
print('start')
rs = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
n_fold = 1
train=[]
train_indices=[]
test=[]
test_indices=[]
for onetrain_indices, onetest_indices in rs.split(user_and_label):
    print(n_fold)
    train_indices.append(onetrain_indices)
    print("train_indices:", onetrain_indices)
    x_train = x_train_names_all[onetrain_indices, ...]
    print("x_train_names:\n", x_train)
    train.append(x_train)

    test_indices.append(onetest_indices)
    print("test_indices:", onetest_indices)
    x_test = x_train_names_all[onetest_indices, ...]
    print("x_test_names:\n", x_test)
    test.append(x_test)

    n_fold+=1

start
1
train_indices: [2223 7572 8499 ... 4859 3264 2732]
x_train_names:
 [ 405006  534821  547287 ... 1329806 1156279 1097647]
test_indices: [6449 9394 4069 5477 9378 8122 5314 3597 8229 6455 1167 4053 8695 6602
 4134 8877 1559 6689  273 1218 1237 2241 7376 3925 8762 4230 6150 5527
 5982  247 4500 8379 1235 2818 7843 7783 2217 6531 2060 5253 8521 1172
 2214 7648 6208 5567 1761 8351 8350 1984 8682 5168 2076 7201 3790 5922
 1132 4726 6153 5381 3964 4221 3614 4320 4271 7580 4905 2384 1399 6028
 4527 8507 7737 7494   33 2967 5436 3101 6746 1654  779 5810 7700 5038
 4763 8508 8346  454 2847 1379 1675 9088 6999 5992 5164 1166   39 2343
 8627  938 4764 2529 1928 8410 7637 2839 3829 7328 4937  771 6377 1085
   31 8504 3921  390 8966 4499 8320 7117   36 8258  776 1373 4071 1812
 8692 5407 3631 6719 7257 4734 2119 8786 1453 5591  296 1946 6299 3645
 9051 3473 1231 5548 4388 3382 1065 5706  154 3197 8851 2250 8753 6353
 7684 3585 8503 5861 4858 1931 5121 7347 2379 8254 4627 2183 3782 6068
 1710

In [53]:
for i in range(10):
    print(i)
    b=user_and_label.iloc[train_indices[i]]
    b.to_csv('/data/jupyter_root/LSK/train+test/feature_label_list_train'+str(i)+'.csv',index=None,header=True)
    b=user_and_label.iloc[test_indices[i]]
    b.to_csv('/data/jupyter_root/LSK/train+test/feature_label_list_test'+str(i)+'.csv',index=None,header=True)

0
1
2
3
4
5
6
7
8
9


# 训练与打印结果

In [106]:
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import numpy as np
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

#测试输出PR
def get_PR(pred,y,th):
    pred=pred[:,1]
    #print(pred)
    pred[pred>th]=1
    pred[pred<th]=0
    #print(pred)
    TP=0
    FP=0
    TN=0
    FN=0
    for i,[j] in enumerate(y):
        #print(i,j,pred[i])
        #break
        if(j==1 and pred[i]==1):
            TP+=1
        if(j==0 and pred[i]==0):
            TN+=1
        if(j==1 and pred[i]==0):
            FN+=1
        if(j==0 and pred[i]==1):
            FP+=1
    P=float(TP+FN)
    N=float(TN+FP)
    print('TP',TP,'P',P,'FP',FP,'N',N)
    Precision = (TP/P)/(TP/P+FP/N)
    Recall=TP/P
    print('Precision ',Precision,'Recall',Recall)
    return Precision ,Recall

Precision1 ,Recall1=[],[]
Precision2 ,Recall2=[],[]
Precision3 ,Recall3=[],[]
Precision4 ,Recall4=[],[]
feature_importance_array=np.array([[0]*70])
model_list=[]
for i in range(10):
    print(i)
    x_train=pd.read_csv('/data/jupyter_root/LSK/train+test/feature_label_list_train'+str(i)+'.csv').drop(['user_name','label'],axis=1)
    y_train=pd.read_csv('/data/jupyter_root/LSK/train+test/feature_label_list_train'+str(i)+'.csv')[['label']]

    x_test=pd.read_csv('/data/jupyter_root/LSK/train+test/feature_label_list_test'+str(i)+'.csv').drop(['user_name','label'],axis=1)
    y_test=pd.read_csv('/data/jupyter_root/LSK/train+test/feature_label_list_test'+str(i)+'.csv')[['label']]
    
    
    #x_train.fillna(-1, inplace=True)
    #x_test.fillna(-1, inplace=True)
    #gbdt_model = GradientBoostingClassifier(n_estimators=500, subsample=0.8, max_features=0.7)
    #gbdt_model.fit(x_train, y_train)
    #pred=gbdt_model.predict_proba(x_test)
    #try:
     #   get_PR(pred.copy(),np.array(y_test),0.7)
    #except:
    #    print("false")
    #get_PR(pred.copy(),np.array(y_test),0.6)
    #get_PR(pred.copy(),np.array(y_test),0.5)
    #get_PR(pred.copy(),np.array(y_test),0.4)    
      
    
    #rf_model = RandomForestClassifier(n_estimators=3000, max_depth=8, max_features='sqrt', oob_score=True)
    #rf_model.fit(x_train, y_train)
    
    #pred=rf_model.predict_proba(x_test)#不能处理空值

      
    
      
    xgb_model = xgb.XGBClassifier(n_estimators=5000,min_child_weight=4,scale_pos_weight=1,subsample=0.5,colsample_bytree=0.8,
                                  learning_rate=0.5, objective = 'binary:logistic', eval_metric='error',alpha = 4e-5,n_jobs=8)
    
    xgb_model.fit(x_train, y_train, verbose = 50, 
                      eval_set=[(x_train, y_train),(x_test, y_test)], early_stopping_rounds = 500,
                     eval_metric = 'error')
    model_list.append(xgb_model)
    feature_importance = xgb_model.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    feature_importance_array=np.concatenate((feature_importance_array,[feature_importance]),axis=0)
    
    
    pred=xgb_model.predict_proba(x_test)
    
    try:
        P,R=get_PR(pred.copy(),np.array(y_test),0.7)
        Precision1.append(P)
        Recall1.append(R)
    except:
        print("false")
    try:
        P,R=get_PR(pred.copy(),np.array(y_test),0.6)
        Precision2.append(P)
        Recall2.append(R)
    except:
        print("false")
    try:
        P,R=get_PR(pred.copy(),np.array(y_test),0.5)
        Precision3.append(P)
        Recall3.append(R)
    except:
        print("false")
    try:
        P,R=get_PR(pred.copy(),np.array(y_test),0.4)
        Precision4.append(P)
        Recall4.append(R)
    except:
        print("false")
print('Precision1',np.array(Precision1).mean() ,'Recall1',np.array(Recall1).mean())
print('Precision2',np.array(Precision2).mean()  ,'Recall2',np.array(Recall2).mean())
print('Precision3',np.array(Precision3).mean()  ,'Recall3',np.array(Recall3).mean())
print('Precision4',np.array(Precision4).mean() ,'Recall4',np.array(Recall4).mean())

temp=feature_importance_array.mean(axis=0)
print(temp/max(temp))
# lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=100, reg_alpha=3, reg_lambda=5, max_depth=-1,
#     n_estimators=5000, objective='binary', subsample=0.9, colsample_bytree=0.77, subsample_freq=1, learning_rate=0.05,
#     random_state=1000, n_jobs=16, min_child_weight=4, min_child_samples=5, min_split_gain=0)
# lgb_model.fit(x_train, y_train, verbose=50,
#                   eval_set=[(x_train, y_train),
#                             (x_test, y_test)], early_stopping_rounds=30)
# pred=lgb_model.predict_proba(x_test, num_iteration=lgb_model.best_iteration_)
# 
# try:
#     get_PR(pred.copy(),np.array(y_test),0.7)
# except:
#     print("false")
# get_PR(pred.copy(),np.array(y_test),0.6)
# get_PR(pred.copy(),np.array(y_test),0.5)
# get_PR(pred.copy(),np.array(y_test),0.4)


0
[0]	validation_0-error:0.298772	validation_1-error:0.312434
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 500 rounds.


/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[50]	validation_0-error:0.284719	validation_1-error:0.301807
[100]	validation_0-error:0.277752	validation_1-error:0.306057
[150]	validation_0-error:0.269603	validation_1-error:0.300744
[200]	validation_0-error:0.260392	validation_1-error:0.311371
[250]	validation_0-error:0.256731	validation_1-error:0.311371
[300]	validation_0-error:0.252362	validation_1-error:0.321998
[350]	validation_0-error:0.245394	validation_1-error:0.321998
[400]	validation_0-error:0.24256	validation_1-error:0.328374
[450]	validation_0-error:0.239017	validation_1-error:0.333688
[500]	validation_0-error:0.235239	validation_1-error:0.33475
[550]	validation_0-error:0.232641	validation_1-error:0.33475
[600]	validation_0-error:0.232404	validation_1-error:0.347503
Stopping. Best iteration:
[112]	validation_0-error:0.274799	validation_1-error:0.290117

TP 269 P 532.0 FP 78 N 409.0
Precision  0.7261297412171571 Recall 0.5056390977443609
TP 376 P 532.0 FP 134 N 409.0
Precision  0.6832657993886401 Recall 0.706766917293233
T

[350]	validation_0-error:0.250709	validation_1-error:0.308183
[400]	validation_0-error:0.247756	validation_1-error:0.319872
[450]	validation_0-error:0.241143	validation_1-error:0.329437
[500]	validation_0-error:0.243033	validation_1-error:0.330499
Stopping. Best iteration:
[31]	validation_0-error:0.285782	validation_1-error:0.284803

TP 259 P 565.0 FP 71 N 376.0
Precision  0.7082524236539902 Recall 0.4584070796460177
TP 397 P 565.0 FP 125 N 376.0
Precision  0.6788269053238561 Recall 0.7026548672566372
TP 481 P 565.0 FP 184 N 376.0
Precision  0.6349924161563957 Recall 0.8513274336283185
TP 512 P 565.0 FP 224 N 376.0
Precision  0.6033497141710962 Recall 0.9061946902654867
7
[0]	validation_0-error:0.302551	validation_1-error:0.27949
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 500 rounds.
[50]	validation_0-error:0.284483	validation_1-error:0.276302
[100]	validation_0-error:0.273855	val

In [107]:
np.argsort(temp)

array([42, 38,  8, 39, 56,  4, 20, 41, 66, 34, 12, 16,  1, 37,  0, 58,  3,
       15, 50, 24, 43, 13, 65, 40, 36, 35, 49, 61, 55, 68,  9, 57,  2, 26,
        5, 18, 62, 31, 63, 33, 69, 64, 28, 17, 10, 23, 47, 52, 19, 25, 59,
       29, 21, 30, 54, 32, 22, 44, 51, 11,  6, 53, 45, 27,  7, 67, 14, 46,
       48, 60])

In [115]:
print(np.sort(temp/max(temp)))

[0.33368873 0.3429583  0.34363403 0.34718989 0.3472386  0.34789576
 0.35903123 0.35918914 0.35919854 0.36028861 0.36138699 0.36727932
 0.36776262 0.36780848 0.36851091 0.3688515  0.36888499 0.37057209
 0.37138994 0.37310789 0.37349903 0.37639845 0.3764334  0.37808924
 0.37816352 0.37820101 0.37835528 0.37857267 0.37859748 0.37913141
 0.37925516 0.37941991 0.38063272 0.38101807 0.38115978 0.3833267
 0.38340907 0.38353017 0.38390013 0.38997385 0.39097313 0.39116529
 0.3912731  0.39141062 0.39284983 0.3934304  0.39365577 0.39628854
 0.40194445 0.40668109 0.40743372 0.41387863 0.41523984 0.41805536
 0.41822219 0.41953563 0.42023615 0.42585713 0.43933979 0.44582637
 0.44741478 0.45301812 0.45380904 0.46440953 0.48401714 0.48477277
 0.56741064 0.58667076 0.63626265 1.        ]


In [118]:
np.array(user_and_label.columns)[14+2]

'no_name_feature5block_1_label'

In [38]:
def get_all_have_name_label(f_count,choice):#f_count：the index of feature;j: index of test; choice: index of block
    
    #准备用于读取数据的 pandas头（防止第一行数据成为标签）
    data_path="/data/jupyter_root/dcube_data/raw_have_name/feature"#feature文件位置
    try:
        cols = pd.read_csv(data_path+str(f_count)+"/block_1.tuples",nrows=1).columns#先读第一行，用于提取数据维度
    except:
        print("false1")
        return 0,0,0        

    user_index=['user_name']#初始化一个等待补全的names
    #print(cols.shape[0]-1)
    for i in range(1,cols.shape[0]-1):
            user_index.append(str(i))
    user_index.append('count')#补上最后的count
    
    #开始读取数据
    #result=pd.read_csv(data_path+str(f_count)+"/test"+str(j)+"/block_1.tuples",sep=',',names=user_index)#读取全部的数据，并给上names
    #try:
    if(choice=='all'):#读取blocks.txt
        result=pd.read_csv(data_path+str(f_count)+"/blocks.txt",sep=',',names=user_index)
    elif(choice==12 or choice==13 or choice==23 or choice ==123 ):#读取blockchoice.txt
        result=pd.read_csv(data_path+str(f_count)+"/block"+str(choice)+".txt",sep=',',names=user_index)
    else:
        result=pd.read_csv(data_path+str(f_count)+"/block_"+str(choice)+".tuples",sep=',',names=user_index)
    #except:
    #    print("false2")
    #    return 0
    find_names = result['user_name'].drop_duplicates()#预测出坏用户的用户列表
    #print(names)
    
    dataset = pd.read_csv(open('/data/jupyter_root/PJT/all_ed_12.csv','r',encoding = 'gb18030'))#读取总标签数据
    feature_label = dataset[['user_name']].drop_duplicates(inplace=False)
    feature_label['label']=np.nan

    
    feature_data=pd.read_csv(data_path+str(f_count)+".txt",sep=',',names=user_index)#读取全部的原始数据，并给上names
    feature_names = feature_data['user_name'].drop_duplicates()#原始数据中用户的用户列表
    
    feature_label.loc[feature_label['user_name'].isin(feature_names),'label']=0
    feature_label.loc[feature_label['user_name'].isin(find_names),'label']=1
    feature_label.rename(columns={'label':'have_name_feature'+str(f_count)+"block_"+str(choice)+'_label'},inplace=True)

    return feature_label

In [37]:
import re

def get_set(s):
    s=re.sub('[ {}]','',s).split(',')
    def func(x):
        return int(float(x))
    return set(map(func,s))

def get_all_no_name_label(result_list,f_count,choice):
    
    dataset = pd.read_csv(open('/data/jupyter_root/PJT/all_ed_12.csv','r',encoding = 'gb18030'))#读取总标签数据
    feature_label = dataset[['user_name']].drop_duplicates(inplace=False)
    feature_label['label']=np.nan
    #print(data)

    feature_names = get_set(result_list.iloc[f_count,0])
    find_names = get_set(result_list.iloc[f_count,choice])
    
    feature_label.loc[feature_label['user_name'].isin(feature_names),'label']=0
    feature_label.loc[feature_label['user_name'].isin(find_names),'label']=1
    feature_label.rename(columns={'label':'no_name_feature'+str(f_count)+"block_"+str(choice)+'_label'},inplace=True)
    
    
    return feature_label

In [35]:
#raw_have_name
have_name=[[1,1],[1,2],[1,3],[1,5],[2,1],[2,3],[2,4],[2,5],
                      [4,1],[4,2],[4,4],[5,1],[5,2],[5,3],[5,4],
                      [6,2],[6,3],[6,5],[7,2],[7,4],
                      [8,2],[8,3],[8,5],[9,5]]

have_name_single=list(range(10,20))

#raw_no_name
no_name=[[0,2],[1,2],[1,3],[1,4],[1,5],[2,1],
         [3,1],[3,2],[3,3],[3,4],[3,5],[4,1],[4,2],[4,5],
                   [5,1],[5,2],[5,3],[5,4],[6,1],[6,2],[6,3],
                   [7,1],[7,2],[7,3],[7,5],[8,1],[8,2],[8,3],[8,4],[8,5],
                   [9,2],[9,3],[9,4],
                   [10,1],[10,2],[10,4]]


In [39]:
import pandas as pd
import csv
import numpy

dataset = pd.read_csv(open('/data/jupyter_root/PJT/all_ed_12.csv','r',encoding = 'gb18030'))#读取总标签数据
all_feature = dataset[['user_name']].drop_duplicates(inplace=False)

#合成no_name特征
result_list=pd.read_csv("/data/jupyter_root/dcube_data/raw_no_name/result_list.csv")
j=0
for x in no_name:
    feature_label=get_all_no_name_label(result_list,x[0],x[1])
    all_feature=all_feature.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    j+=1
    print('done'+str(j))
    
#合成have_name特征
for x in have_name:
    feature_label=get_all_have_name_label(x[0],x[1])
    all_feature=all_feature.merge(feature_label,left_on='user_name',right_on='user_name',how='left')
    j+=1
    print('done'+str(j))

#读取含user_name的一维度的add的特征
for i in have_name_single:
    cols = pd.read_csv('/data/jupyter_root/dcube_data/raw_have_name/feature'+str(i)+'.txt',nrows=1).columns
    user_index=['user_name']#初始化一个等待补全的names
    user_index.append('have_name_feature'+str(i)+'count')#补上最后的count
    oneadd=pd.read_csv('/data/jupyter_root/dcube_data/raw_have_name/feature'+str(i)+'.txt',sep=',',names=user_index)
    all_feature=all_feature.merge(oneadd,left_on='user_name',right_on='user_name',how='left')
    j+=1
    print('done'+str(j))

all_feature.to_csv('/data/jupyter_root/LSK/raw_all_feature.csv',index=None,header=True)
all_feature

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


done1
done2
done3
done4
done5
done6
done7
done8
done9
done10
done11
done12
done13
done14
done15
done16
done17
done18
done19
done20
done21
done22
done23
done24
done25
done26
done27
done28
done29
done30
done31
done32
done33
done34
done35
done36
done37
done38
done39
done40
done41
done42
done43
done44
done45
done46
done47
done48
done49
done50
done51
done52
done53
done54
done55
done56
done57
done58
done59
done60
done61
done62
done63
done64
done65
done66
done67
done68
done69
done70


,user_name,no_name_feature0block_2_label,no_name_feature1block_2_label,no_name_feature1block_3_label,no_name_feature1block_4_label,no_name_feature1block_5_label,no_name_feature2block_1_label,no_name_feature3block_1_label,no_name_feature3block_2_label,no_name_feature3block_3_label,...,have_name_feature10count,have_name_feature11count,have_name_feature12count,have_name_feature13count,have_name_feature14count,have_name_feature15count,have_name_feature16count,have_name_feature17count,have_name_feature18count,have_name_feature19count
0,1556152.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,2,2,2,1,10,6,6
1,366589.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,4,4,4,4,1,3,2,1
2,1348648.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5,3,2,2,2,2,1,46,16,18
3,7528.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,2,1,2,2,2
4,1565767.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
5,493329.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
6,1106770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,2,1,1,2,2,1,2,2,2
7,478360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,2,2,2,2,1,2,2,2
8,1406615.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,2,2,1
9,1589558.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2,2,2,2,2,2,1,2,2,2


In [ ]:
#删除空行
#all_feature=all_feature.drop(range(9409,9502))
all_feature.to_csv('/data/jupyter_root/LSK/raw_all_feature.csv',index=None,header=True)
all_feature

In [61]:
x_pred=pd.read_csv('/data/jupyter_root/LSK/raw_all_feature.csv').drop(['user_name'],axis=1)
y_pred=None
for model in model_list:
    if(id(y_pred)==id(None)):
        y_pred=model.predict_proba(x_pred)[:,1]
    else:
        y_pred+=model.predict_proba(x_pred)[:,1]
y_pred=y_pred/10.0
y_pred

array([0.8157218 , 0.66930246, 0.03658939, ..., 0.7435133 , 0.7191825 ,
       0.7191825 ], dtype=float32)

In [68]:
raw_result= all_feature[['user_name']].drop_duplicates(inplace=False)
raw_result['score']=y_pred
all_user = pd.read_csv('/data/jupyter_root/PJT/raw_dataset.csv')[['user_name']].drop_duplicates(inplace=False)
raw_result=raw_result.merge(all_user,left_on='user_name',right_on='user_name',how='right')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [132]:
raw_result.fillna(0)
raw_result['label(th=0.5)']=0
raw_result.loc[raw_result['score']>0.5,'label(th=0.5)']=1

In [133]:
raw_result['label(th=0.7)']=0
raw_result.loc[raw_result['score']>0.7,'label(th=0.7)']=1

In [134]:
raw_result['label(th=0.75)']=0
raw_result.loc[raw_result['score']>0.75,'label(th=0.75)']=1

In [130]:
raw_result.drop_duplicates(inplace=True)
raw_result.to_csv('/data/jupyter_root/LSK/raw_result.csv',index=None,header=True)
print(raw_result['label(th=0.5)'].sum())
print(raw_result['label(th=0.7)'].sum())
print(raw_result['label(th=0.75)'].sum())

293370
106814
78260


In [ ]:
#raw_result=pd.read_csv('/data/jupyter_root/LSK/raw_result_ed.csv')
raw_result.drop_duplicates(inplace=True)
print(raw_result['label(th=0.5)'].sum())
print(raw_result['label(th=0.7)'].sum())
print(raw_result['label(th=0.75)'].sum())

In [126]:
raw_result=pd.read_csv('/data/jupyter_root/GSD/raw_result.csv')
raw_result.drop_duplicates(inplace=True)
print(raw_result['label(th=0.5)'].sum())
print(raw_result['label(th=0.7)'].sum())
print(raw_result['label(th=0.75)'].sum())

343730
114878
83141


In [129]:
temp=user_and_label[['user_name','label']].merge(raw_result,left_on='user_name',right_on='user_name',how='left')
def get_PR_temp(pred,y):
    #print(pred)
    TP=0
    FP=0
    TN=0
    FN=0
    for i,j in enumerate(y):
        #print(i,j,pred[i])
        #break
        if(j==1 and pred[i]==1):
            TP+=1
        if(j==0 and pred[i]==0):
            TN+=1
        if(j==1 and pred[i]==0):
            FN+=1
        if(j==0 and pred[i]==1):
            FP+=1
    P=float(TP+FN)
    N=float(TN+FP)
    print('TP',TP,'P',P,'FP',FP,'N',N)
    Precision = (TP/P)/(TP/P+FP/N)
    Recall=TP/P
    print('Precision ',Precision,'Recall',Recall)
    return Precision ,Recall
get_PR_temp(np.array(temp['label(th=0.75)']),np.array(temp['label']))
get_PR_temp(np.array(temp['label(th=0.7)']),np.array(temp['label']))
get_PR_temp(np.array(temp['label(th=0.5)']),np.array(temp['label']))

TP 1535 P 5538.0 FP 400 N 3871.0
Precision  0.7284357287471106 Recall 0.2771758757674251
TP 1935 P 5538.0 FP 538 N 3871.0
Precision  0.7154257247181401 Recall 0.3494041170097508
TP 3536 P 5538.0 FP 1264 N 3871.0
Precision  0.6616362192216044 Recall 0.6384976525821596


(0.6616362192216044, 0.6384976525821596)

In [ ]:
TP 2183 P 5538.0 FP 364 N 3871.0
Precision  0.8073964586085242 Recall 0.3941856265799928
TP 2945 P 5538.0 FP 611 N 3871.0
Precision  0.7711200757206548 Recall 0.5317804261466234
TP 4874 P 5538.0 FP 1919 N 3871.0
Precision  0.6396833787901247 Recall 0.8801011195377393

#现在
TP 2164 P 5538.0 FP 348 N 3871.0
Precision  0.8129647436332913 Recall 0.39075478512098233
TP 2898 P 5538.0 FP 576 N 3871.0
Precision  0.7786037051797309 Recall 0.52329360780065
TP 4780 P 5538.0 FP 1771 N 3871.0
Precision  0.6535715327705545 Recall 0.8631274828457927

#GSD
TP 1516 P 5538.0 FP 411 N 3871.0
Precision  0.720534973431326 Recall 0.27374503430841457
TP 1928 P 5538.0 FP 562 N 3871.0
Precision  0.7057052979468673 Recall 0.3481401227880101
TP 3583 P 5538.0 FP 1384 N 3871.0
Precision  0.6440765779937528 Recall 0.6469844709281329

#现在
TP 1535 P 5538.0 FP 400 N 3871.0
Precision  0.7284357287471106 Recall 0.2771758757674251
TP 1935 P 5538.0 FP 538 N 3871.0
Precision  0.7154257247181401 Recall 0.3494041170097508
TP 3536 P 5538.0 FP 1264 N 3871.0
Precision  0.6616362192216044 Recall 0.6384976525821596